In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import models

In [ ]:
class myVGG(nn.Module):

    def __init__(self):
        super(myVGG, self).__init__()

        self.conv01 = nn.Conv2d(3, 64, 3)
        self.conv02 = nn.Conv2d(64, 64, 3)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv03 = nn.Conv2d(64, 128, 3)
        self.conv04 = nn.Conv2d(128, 128, 3)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv05 = nn.Conv2d(128, 256, 3)
        self.conv06 = nn.Conv2d(256, 256, 3)
        self.conv07 = nn.Conv2d(256, 256, 3)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv08 = nn.Conv2d(256, 512, 3)
        self.conv09 = nn.Conv2d(512, 512, 3)
        self.conv10 = nn.Conv2d(512, 512, 3)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.conv11 = nn.Conv2d(512, 512, 3)
        self.conv12 = nn.Conv2d(512, 512, 3)
        self.conv13 = nn.Conv2d(512, 512, 3)
        self.pool5 = nn.MaxPool2d(2, 2)

        self.avepool1 = nn.AdaptiveAvgPool2d((7, 7))

        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 4)

        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)



    def forward(self, x):
        x = F.relu(self.conv01(x))
        x = F.relu(self.conv02(x))
        x = self.pool1(x)

        x = F.relu(self.conv03(x))
        x = F.relu(self.conv04(x))
        x = self.pool2(x)

        x = F.relu(self.conv05(x))
        x = F.relu(self.conv06(x))
        x = F.relu(self.conv07(x))
        x = self.pool3(x)

        x = F.relu(self.conv08(x))
        x = F.relu(self.conv09(x))
        x = F.relu(self.conv10(x))
        x = self.pool4(x)

        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.pool5(x)

        x = self.avepool1(x)

        # 行列をベクトルに変換
        x = x.view(-1, 512 * 7 * 7)
        
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        return x

In [ ]:
transform_train = torchvision.transforms.Compose([
     torchvision.transforms.CenterCrop(224),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_valid = torchvision.transforms.Compose([
     torchvision.transforms.CenterCrop(224),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


train_data_dir = 'drive/MyDrive/sprit_data/train'
valid_data_dir = 'drive/MyDrive/sprit_data/validation'

# training set
trainset = torchvision.datasets.ImageFolder(train_data_dir, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)

# validation set
validset = torchvision.datasets.ImageFolder(valid_data_dir, transform=transform_valid)
validloader = torch.utils.data.DataLoader(validset, batch_size=16, shuffle=False)

In [ ]:
net = myVGG()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
net.train()


myVGG(
  (conv01): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv02): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv03): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv04): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv05): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv06): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv07): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv08): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv09): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv10): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv11): Con

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001, momenton=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
# 同じデータを 50 回学習します
for epoch in range(32):

  # 今回の学習効果を保存するための変数
  running_loss = 0.0

  for data in trainloader:
    # データ整理
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    # 前回の勾配情報をリセット
    optimizer.zero_grad()

    # 予測
    outputs = net(inputs)

    # 予測結果と教師ラベルを比べて損失を計算
    loss = criterion(outputs, labels)
    running_loss += loss.item()

    # 損失に基づいてネットワークのパラメーターを更新
    loss.backward()
    optimizer.step()

  # このエポックの学習効果
  print("Epoch:",epoch+1)
  print('-'*50)
  print(running_loss)
  scheduler.step()

KeyboardInterrupt: ignored

In [ ]:
# モデルを評価モードにする
net.eval()

# 全検証データの正しく分類できた枚数を記録
n_correct = 0
n_total = 0

for data in validloader:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    # 予測
    outputs = net(inputs)

    # 予測結果をクラス番号に変換
    _, predicted = torch.max(outputs.data, 1)
    
    # 予測結果と実際のラベルを比較して、正しく予測できた枚数を計算
    res = (predicted == labels)
    res = res.sum().item()
    

    # 今までに正しく予測できた枚数に計上
    n_correct = n_correct + res
    n_total = n_total + len(labels)


print(n_correct / n_total)


0.39655172413793105
